In [112]:
import pandas as pd
import numpy as np
from helper_funcs import *

In [113]:
data = pd.read_csv('https://nycdsacapstone2021.blob.core.windows.net/fromsmh/Zip_Master_File_Demographics.csv', dtype={'CountyFIPS':'string', 'StateFIPS':'string'})

In [114]:
data.columns

Index(['ZipCode', 'PrimaryRecord', 'Population', 'HouseholdsPerZipCode',
       'WhitePopulation', 'BlackPopulation', 'HispanicPopulation',
       'AsianPopulation', 'HawaiianPopulation', 'IndianPopulation',
       'OtherPopulation', 'MalePopulation', 'FemalePopulation',
       'PersonsPerHousehold', 'AverageHouseValue', 'IncomePerHousehold',
       'MedianAge', 'MedianAgeMale', 'MedianAgeFemale', 'Latitude',
       'Longitude', 'Elevation', 'State', 'StateFullName', 'CityType',
       'CityAliasAbbreviation', 'AreaCode', 'City', 'CityAliasName', 'County',
       'CountyFIPS', 'StateFIPS', 'TimeZone', 'DayLightSaving', 'MSA', 'PMSA',
       'CSA', 'CBSA', 'CBSA_Div', 'CBSAType', 'CBSA_Name', 'MSA_Name',
       'PMSA_Name', 'Region', 'Division', 'MailingName', 'NumberOfBusinesses',
       'NumberOfEmployees', 'BusinessFirstQuarterPayroll',
       'BusinessAnnualPayroll', 'BusinessEmploymentFlag', 'GrowthRank',
       'GrowingCountiesA', 'GrowingCountiesB', 'GrowthIncreaseNumber',
      

In [115]:
data = data[['ZipCode','State','County','CBSA_Name','CountyFIPS', 'StateFIPS', 'Population', 'HouseholdsPerZipCode',
             'WhitePopulation', 'BlackPopulation', 'HispanicPopulation', 'AsianPopulation', 'IndianPopulation', 
             'OtherPopulation', 'MalePopulation', 'FemalePopulation', 'AverageHouseValue', 'IncomePerHousehold',
             'MedianAge','NumberOfBusinesses']]

In [116]:
print(data.CBSA_Name.nunique())
#data.CBSA_Name.unique()

156


In [117]:
replace_states = ['FL','NC','GA','VA','MD','SC','WV','DC','TN','KY','OH','DE','AL']
for _ in range(4):
    for each in replace_states:
        data.CBSA_Name = data.CBSA_Name.str.replace(each,'', regex=False)
data.CBSA_Name = data.CBSA_Name.str.replace('-',' ', regex=False)
data.CBSA_Name = data.CBSA_Name.str.strip()

In [118]:
data.shape

(6987, 20)

In [119]:
data.to_csv('ZIP_master_file.csv', index=False)

#### More Population info

In [120]:
pop = pd.read_csv('https://nycdsacapstone2021.blob.core.windows.net/additionaldata/census2019_pop_est.csv', 
                  dtype={'ZIP':'string'}, index_col='ZIP')
pop.index = pop.index.astype('int64')

In [121]:
data.set_index('ZipCode', inplace=True)
data = data.merge(pop['over_65_ratio'], how='left', left_on='ZipCode', right_index=True)

#### Schools

In [122]:
schools = get_schools()
schools.columns

----pulling schools data from Azure storage----
----Done----


Index(['name', 'street', 'city', 'zip', 'state', 'lat', 'long', 'districtID',
       'districtName', 'gradeLevels', 'ratingScale', 'enrollment',
       'schoolType', 'pctLowIncome', 'overallRating'],
      dtype='object')

In [123]:
zip_school_rating = schools.groupby('zip').agg("mean")['overallRating']
data = data.merge(zip_school_rating, how='left', left_index=True, right_index=True)
data.rename(columns={'overallRating':'school_rating'}, inplace=True)

#### Hospital Data

In [124]:
hosp = pd.read_csv('Hospitals_geocoded.csv')
hosp = hosp[hosp.Overall_Rating>2]
data = data.merge(hosp.groupby('ZIP_Code').agg('count')['Facility_Name'], how='left', left_index=True, right_index=True)
data.rename(columns={'Facility_Name':'Hosp_count'}, inplace=True)

In [125]:
data.head()

,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,...,OtherPopulation,MalePopulation,FemalePopulation,AverageHouseValue,IncomePerHousehold,MedianAge,NumberOfBusinesses,over_65_ratio,school_rating,Hosp_count
ZipCode,,,,,,,,,,,,,,,,,,,,,
20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,3555,...,1952,18912,19639,692900,116195,30.3,1978,0.078726,5.727273,NaN
20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,2299,...,999,24896,27474,637300,90424,34.9,1674,0.085151,4.851852,NaN
20003,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,26454,11724,15295,10111,1355,...,460,14054,12400,806500,134438,34.1,1227,0.101423,5.416667,NaN
20004,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,1622,1127,1321,131,109,...,27,834,788,594600,146822,37.9,961,0.117005,8.000000,NaN
20005,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,12775,8316,8478,2157,2142,...,1228,7000,5775,536700,93956,32.5,2366,0.090953,6.000000,NaN


#### Bring in MSA level FRED data

In [126]:
msa_data = pd.read_csv('MSA_data.csv')
msa_data = msa_data.set_index('CBSA_Name')
msa_data.head()

,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate
CBSA_Name,,,,,,,,
Albany,21,187.61,49.163880,52.173913,0.9959,45.0,279.0,3.7
Albemarle,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Americus,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arcadia,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Asheville,52,373.46,71.571906,7.023411,1.2686,63.0,106.0,3.6


In [127]:
data.loc[data.CBSA_Name=='Gainesville','CBSA_Name'] = \
    data.loc[data.CBSA_Name=='Gainesville','CBSA_Name'] + '_' + data.loc[data.CBSA_Name=='Gainesville','State']

In [128]:
data.loc[data.CBSA_Name=='Jacksonville','CBSA_Name'] = \
    data.loc[data.CBSA_Name=='Jacksonville','CBSA_Name'] + '_' + data.loc[data.CBSA_Name=='Jacksonville','State']

In [129]:
print(f'ZIP master file MSA name dtype: {data.CBSA_Name.dtype}')
print(f'MSA file dtype: {msa_data.index.dtype}')

ZIP master file MSA name dtype: object
MSA file dtype: object


In [130]:
data = data.merge(msa_data, how='left', left_on='CBSA_Name', right_index=True)

In [131]:
data

,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,...,school_rating,Hosp_count,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate
ZipCode,,,,,,,,,,,,,,,,,,,,,
20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,3555,...,5.727273,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,2299,...,4.851852,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
20003,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,26454,11724,15295,10111,1355,...,5.416667,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
20004,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,1622,1127,1321,131,109,...,8.000000,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
20005,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,12775,8316,8478,2157,2142,...,6.000000,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56972,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,NaN,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
56973,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,NaN,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4
56998,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,NaN,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4


#### Zillow Home Value Forecasts by ZIP

In [133]:
zillow = pd.read_csv('https://nycdsacapstone2021.blob.core.windows.net/additionaldata/ZillowHV_forecast_ZIP.csv')
zillow.rename(columns={'ForecastYoYPctChange':'Zillow_HVF'}, inplace=True)
zillow = zillow[zillow.Region=='Zip']

In [134]:
zillow[['RegionName']] = zillow[['RegionName']].astype('int64')
zillow = zillow.set_index('RegionName')

In [135]:
data = data.merge(zillow.Zillow_HVF, how='left', left_index=True, right_index=True)

#### BEA per capita income

In [181]:
BEA = pd.read_csv('https://nycdsacapstone2021.blob.core.windows.net/additionaldata/BEA_per_cap_income.csv')
BEA = BEA[~BEA.isna().any(axis=1)]
BEA['per_cap_income'] = BEA.per_cap_income.str.replace(',','', regex=False).str.replace('(NA)','0', regex=False)
BEA['per_cap_income'] = BEA['per_cap_income'].astype('int64')
BEA['county_code'] = BEA.county_code.str.upper()
BEA.rename(columns={'per_cap_income':'BEA_percap_income'}, inplace=True)
BEA = BEA[['state_code', 'county_code', 'BEA_percap_income']]

In [187]:
data.reset_index()

,ZipCode,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,...,Hosp_count,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate,Zillow_HVF
0,20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,7.1
1,20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.2
2,20003,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,26454,11724,15295,10111,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.3
3,20004,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,1622,1127,1321,131,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,5.8
4,20005,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,12775,8316,8478,2157,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,56972,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN
6983,56973,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN
6984,56998,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN
6985,56999,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,...,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN


In [189]:
data = data.reset_index().merge(BEA, how='left', left_on=['State','County'], right_on=['state_code','county_code']).\
    drop(['state_code','county_code'], axis=1).set_index('ZipCode')

In [197]:
data = data.reset_index()\
            .drop_duplicates(subset='ZipCode', keep='first')\
            .set_index('ZipCode')

In [198]:
data

,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,...,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate,Zillow_HVF,BEA_percap_income
ZipCode,,,,,,,,,,,,,,,,,,,,,
20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,3555,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,7.1,86567.0
20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,2299,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.2,86567.0
20003,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,26454,11724,15295,10111,1355,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.3,86567.0
20004,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,1622,1127,1321,131,109,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,5.8,86567.0
20005,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,12775,8316,8478,2157,2142,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,6.5,86567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56972,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN,86567.0
56973,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN,86567.0
56998,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,0,0,0,0,0,...,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,NaN,86567.0


In [199]:
data.to_csv('ZIP_all_data.csv')